<a href="https://colab.research.google.com/github/mmsamiei/MS-Thesis-Phase3/blob/master/Modeks/Boroujerdi/Boroujerdi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#In the name of God

In [1]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
  function ClickConnect(){
    console.log("Working"); 
    document.querySelector("colab-connect-button").click() 
  }
  var connect_timer = setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [2]:
!nvidia-smi

Sat Jun  6 17:35:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip -q install transformers

     |████████████████████████████████| 675kB 13.7MB/s 
     |████████████████████████████████| 1.1MB 59.6MB/s 
     |████████████████████████████████| 890kB 56.2MB/s 
     |████████████████████████████████| 3.8MB 56.9MB/s 


In [0]:
from torch.utils.data import Dataset, DataLoader
import os
import torch
import json
from torch.utils.data.sampler import SubsetRandomSampler
from transformers import AutoTokenizer
import random
import pandas as pd
import logging

# Dataset

In [0]:
train_file = '/content/drive/My Drive/Thesis/phase-3/hkr_train.csv'
valid_file =  '/content/drive/My Drive/Thesis/phase-3/hkr_valid.csv'

In [8]:
tokenizer = AutoTokenizer.from_pretrained('google/bert_uncased_L-2_H-128_A-2')

In [0]:
df = pd.read_csv(train_file)

In [10]:
df

,Unnamed: 0,history,knowledge,response
0,0,Science fiction,Science fiction (often shortened to SF or sci-...,I think science fiction is an amazing genre fo...
1,1,Science fiction [SEP] I think science fiction ...,Science fiction films have often been used to ...,Awesome! I really love how sci-fi storytellers...
2,2,Science fiction [SEP] I think science fiction ...,The central premise for these stories oftentim...,"It's not quite sci-fi, but my favorite version..."
3,3,Science fiction [SEP] I think science fiction ...,Science fiction often explores the potential c...,If you really want a look at the potential neg...
4,4,Science fiction [SEP] I think science fiction ...,no_passages_used,It blends science fiction and paranormal/psych...
...,...,...,...,...
41484,41484,Beard [SEP] These days I am trying to grow a b...,The Egyptian priest class ritualistically remo...,Have you considered head shaving as well? Just...
41485,41485,Collie,The collie is a distinctive type of herding do...,Have you ever heard of the Collie dog? Its a h...
41486,41486,Collie [SEP] Have you ever heard of the Collie...,The Border Collie is a working and herding dog...,"One of my favorites is the Border Collie, whic..."
41487,41487,Collie [SEP] Have you ever heard of the Collie...,Their ability to be trained to act on the soun...,I prefer collies cause they can be trained and...


In [0]:
class MyDataset(Dataset):
    """My dataset."""

    def __init__(self, csv_file, frac=1, split_rate=1, bound=False, max_len=512):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """
        self.dialogues = pd.read_csv(csv_file)
        self.dialogues.dropna(inplace=True)
        if bound:
          self.dialogues = self.dialogues[(self.dialogues['history'].apply(len)<280) 
          & (self.dialogues['false_sentence'].apply(len)<280) 
          & (self.dialogues['true_sentence'].apply(len)<280)]
        
        self.dialogues = self.dialogues[self.dialogues.index % split_rate == 0]

        self.dialogues = self.dialogues.sample(frac=frac)
        
        # s = (self.dialogues.true_sentence.str.len() + self.dialogues.history.str.len()).sort_values().index
        # self.dialogues = self.dialogues.reindex(s)
        # s = (self.dialogues.false_sentence.str.len() + self.dialogues.history.str.len()).sort_values().index
        # self.dialogues = self.dialogues.reindex(s)


        self.dialogues.dropna(inplace=True)

        self.max_len = max_len


    def __len__(self):
        return len(self.dialogues)

    @staticmethod
    def truncuate_join_pair_sentence(sentence1, sentence2, max_len=510):

        """
        truncuate sentence one from head and sentence two from tail
        Args:
            sentence1 (string): first sentence
            sentence2 (string): seconde sentence
        """
        temp1 = tokenizer.encode(sentence1,add_special_tokens=False)
        temp2 = tokenizer.encode(sentence2,add_special_tokens=False)
        ### two above line may cause warning but no problem because we've handle them below
        logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
        seq_1 = temp1
        seq_2 = temp2
        num_tokens_to_remove = len(temp1) + len(temp2) + 3 - max_len
        if num_tokens_to_remove > 0 :
            seq_1, seq_2, _ = tokenizer.truncate_sequences(temp1[::-1],temp2, num_tokens_to_remove=num_tokens_to_remove)
            seq_1.reverse()
        result_list = [tokenizer.cls_token_id]+seq_1+[tokenizer.sep_token_id]+seq_2+[tokenizer.sep_token_id]
        token_type_ids = [0] * (len(seq_1) + 2) + [1] * (len(seq_2) + 1)
        return result_list, token_type_ids


    def __getitem__(self, idx):
      
        
        history = self.dialogues.iloc[idx].history
        knowledge = self.dialogues.iloc[idx].knowledge
        response = self.dialogues.iloc[idx].response


        input_pair, input_pair_segments = MyDataset.truncuate_join_pair_sentence(history, knowledge, self.max_len)
        
        response_tensor = tokenizer.encode(response)
        

        input_pair = torch.LongTensor(input_pair)

        input_pair_segments = torch.LongTensor(input_pair_segments)


        sample = {'input_pair': input_pair,
                  'input_pair_segments': input_pair_segments,
                  'response': response_tensor}

        return sample


In [0]:
train_dataset = MyDataset(train_file, max_len=128)

In [23]:
tokenizer.decode(train_dataset[0]['input_pair'])
tokenizer.decode(train_dataset[0]['response'])

"[CLS] i think since transportation existed. i do know someone who's family member used to give people rides on there back from place to place. don't think he would have given me a free ride though. lol. [SEP]"